In [1]:
import os
import pickle
import subprocess
# import pandas as pd
from utils_mocap_viz.generate_views import (    # organize this
    # get_output_dir,
    prepare_videos
)

from utils_mocap_viz.animated_merged_phase_analysis import animate_merged_phase_analysis, animate_merged_phase_analysis_with_user_window
from utils_dance_anim.dance_dot import animate_dance_phase_analysis, animate_dance_phase_analysis_with_user_window
from utils_pipeline.pipeline_B import *
from utils_composite_video.by_time_segment import *


### Config

In [2]:
file_name = "BKO_E1_D1_01_Suku"
traj_dir  = "traj_files_presentation"
status    = "included"   # or "excluded"
traj_threshold = "0.001"        # or any other threshold

bvh_dir = os.path.join("data", "bvh_files")
bvh_file = os.path.join(bvh_dir, file_name + "_T")

# path to onsets and cycles csv files
cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
onsets_csv_path = f"data/drum_onsets/{file_name}.csv"
dance_csv_path = f"data/dance_onsets/{file_name}_T_dance_onsets.csv"


m_idx = 0
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]

motion_data_dir = "data/motion_data_pkl"
dance_mode_path = f"data/dance_modes_ts/{file_name}_{dance_mode}.pkl"

# load dance mode time segments
if os.path.exists(dance_mode_path):
    with open(dance_mode_path, "rb") as f:
        dmode_ts = pickle.load(f)
else:
    print(f"{file_name} {dance_mode} does not exist")


mode_start_time, mode_end_time = dmode_ts[0]

print("Dance Mode:", dance_mode)
print(f"Mode start time: {mode_start_time:.2f}")
print(f"Mode end time: {mode_end_time:.2f}")

Dance Mode: group
Mode start time: 35.52
Mode end time: 71.44


In [3]:
# choose start and end time for the composite video

start_time = 40
end_time = 46

# check if start_time and end_time are within the dance mode time segment
if start_time < mode_start_time or end_time > mode_end_time:
    print(f"Start time {start_time:.2f} or end time {end_time:.2f} is outside the dance mode time segment {mode_start_time:.2f} to {mode_end_time:.2f}")
    raise ValueError("Start time or end time is outside the dance mode time segment")

cyc_df = pd.read_csv(cycles_csv_path)
cycle_onsets = cyc_df["Virtual Onset"].values
cycle_onsets_in_range = cycle_onsets[(cycle_onsets >= start_time) & (cycle_onsets <= end_time)] 


traj_tuples = [ (cycle_onsets_in_range[i], cycle_onsets_in_range[i+1], 0) for i in range(len(cycle_onsets_in_range)-1)]
traj_tuples

base_output_dir = os.path.join("composite_videos", f"{file_name}_{start_time:.2f}_{end_time:.2f}")
os.makedirs(base_output_dir, exist_ok=True)

traj_tuples

[(41.308333333, 43.290555555, 0), (43.290555555, 45.284333333, 0)]

### Generate trajectory video + trimmed dance video plots

In [4]:
extract_forward_cycle_videos_and_plots(
    file_name = file_name,
    windows = traj_tuples,  # List of (win_start, win_end, t_poi) tuples
    base_path_logs = "data/logs_v4_0.007_foot_jun3",            # logs_v4_0.007_foot_jun3       logs_v2_may
    figsize = (10, 3),
    dpi = 200,
    save_dir = base_output_dir,
    legend_flag = False,
    )

Windows data:
Window 1:
  Start: 41.308
  End: 43.291
  Duration: 1.982
Window 2:
  Start: 43.291
  End: 45.284
  Duration: 1.994

Foot data ranges:
Left foot time range: 34.258 to 445.629
Right foot time range: 35.304 to 446.496
Number of left foot onsets: 354
Number of right foot onsets: 358

Processing 2 windows
Total frames in trajectory data: 107339
Time range in trajectory data: 0.000 to 447.242

Processing window 1:
  Window time range: 41.308 to 43.291
  Found 1 left foot onsets and 1 right foot onsets
  Left foot onset times: [42.3]
  Right foot onset times: [41.33333333]
  Video frames: 2065 to 2164 (50fps)
  Trajectory frames: 9913 to 10389 (240fps)
  Trajectory data points: 476

Video extraction:
Input video: data/videos/BKO_E1_D1_01_Suku_pre_R_Mix.mp4
Output video: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\videos\BKO_E1_D1_01_Suku_window_001_41.31_43.29.mp4
Start time: 41.308333333
Duration: 1.9822222219999972
FFmpeg command: ffmpeg -y -i data/videos/BKO_E1_D1_01_Suku

### Generate Skeletal video + trimmed_video_mix + drum dot

In [5]:
# video_path = os.path.join("data", "videos", f"{file_name}_pre_R_Mix.mp4")

output_dir1 = os.path.join(base_output_dir, "video_skeleton")
os.makedirs(output_dir1, exist_ok=True)

output_dir3 = os.path.join(base_output_dir, "drum_dot_merged")
os.makedirs(output_dir3, exist_ok=True)

output_dir4 = os.path.join(base_output_dir, "dance_dot")
os.makedirs(output_dir4, exist_ok=True)

views_to_generate = ['front']       # skeleton views ['front', 'right' 'left', 'top'] 

for c_start_time, c_end_time, _ in traj_tuples:

    # Drum dot plot video
    animate_merged_phase_analysis_with_user_window(
        file_name= file_name,
        W_start= mode_start_time,   # dance mode start time
        W_end= mode_end_time,       # dance mode end time
        user_start= c_start_time,     # user defined start time
        user_end= c_end_time,         # user defined end time
        cycles_csv_path= cycles_csv_path,
        onsets_csv_path= onsets_csv_path,
        save_dir=output_dir3,
        figsize=(10, 3),
        dpi=200,
        legend_flag=False
    )


    # Dance dot plot video
    animate_dance_phase_analysis_with_user_window(
        file_name= file_name,
        W_start= mode_start_time,   # dance mode start time
        W_end= mode_end_time,       # dance mode end time
        user_start= c_start_time,     # user defined start time
        user_end= c_end_time,         # user defined end time
        cycles_csv_path= cycles_csv_path,
        dance_csv_path= dance_csv_path,
        save_dir=output_dir4,
        figsize=(10, 3),
        dpi=200,
    )
    
    
    # Generate Skeleton views
    view_videos = prepare_videos(
        filename= bvh_file,
        start_time= c_start_time,
        end_time= c_end_time,
        views_to_generate = views_to_generate,
        video_path= None,             # video_path, wont generate video
        video_size= (1280, 720),
        fps= 24,
        output_dir = output_dir1
    )
    

Generating animation for drum merged dot plot for BKO_E1_D1_01_Suku
Full window: 35.5s - 71.4s
Animation window: 41.3s - 43.3s
Total onsets: 2821
Onsets in window 35.52s - 71.44s: 105
Total onsets: 2821
Onsets in window 35.52s - 71.44s: 151
Total onsets: 2821
Onsets in window 35.52s - 71.44s: 151

Creating animation...
Animation will have 48 frames
Time range: 41.31s - 43.27s

Saving animation to: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\drum_dot_merged\drum_dot_merged_41.31_43.29.mp4
Animation saved successfully!
Generating animation for dance dot plot for BKO_E1_D1_01_Suku
Full window: 35.5s - 71.4s
Animation window: 41.3s - 43.3s

Creating animation...
Animation will have 48 frames
Time range: 41.31s - 43.27s

Saving animation to: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\dance_dot\dance_41.31_43.29.mp4
Animation saved successfully!
Generating Skeleton views for data\bvh_files\BKO_E1_D1_01_Suku_T | Window: 41.3s - 43.3s

Generating front view...

BVH file information:
Fra

### Generate animated kinematic plots

In [6]:
# Available markers: ['Hips', 'LeftHip', 'LeftKnee', 'LeftAnkle', 'LeftToe', 
# 'LeftToeEnd', 'RightHip', 'RightKnee', 'RightAnkle', 'RightToe', 'RightToeEnd', 
# 'Chest', 'Chest2', 'Chest3', 'Chest4', 'LeftCollar', 'LeftShoulder', 'LeftElbow', 
# 'LeftWrist', 'LeftWristEnd', 'RightCollar', 'RightShoulder', 'RightElbow', 
# 'RightWrist', 'RightWristEnd', 'Neck', 'Head', 'HeadEnd']

mvnx_to_bvh = {
    'x': 'z',  # forward mvnx → backward bvh
    'y': 'x',  # side mvnx → side bvh
    'z': 'y',  # vertical mvnx → vertical bvh
}

joint_name = "Hips"  
axis = 'z'      # z is vertical in mvnx files

output_dir2 = os.path.join(base_output_dir, joint_name)
os.makedirs(output_dir2, exist_ok=True)

extract_kinematic_cycle_plots(
file_name= file_name,
windows= traj_tuples,
joint_name= joint_name,
axis= mvnx_to_bvh[axis],
output_dir2= output_dir2,
figsize = (10, 3),  # 2000 x 600 px
dpi= 200,
legend_flag = False,
)

Successfully loaded CSV with 107339 rows

Processing 2 windows

Processing window 1:
  Window time range: 41.308 to 43.291
  Trajectory frames: 9913 to 10389 (240fps)
  Trajectory data points: 476
Plot saved: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\Hips\BKO_E1_D1_01_Suku_window_001_41.31_43.29.mp4
Plot duration: 2.000s

Processing window 2:
  Window time range: 43.291 to 45.284
  Trajectory frames: 10389 to 10868 (240fps)
  Trajectory data points: 479
Plot saved: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\Hips\BKO_E1_D1_01_Suku_window_002_43.29_45.28.mp4
Plot duration: 2.000s

Processing complete!


### Prepare for concatenation

In [7]:
concatenate_and_overlay_videos(file_name, joint_name, base_output_dir, views_to_generate)        # modify

concat_file_list = [f for f in os.listdir(base_output_dir) if f.lower().endswith(".mp4")]
concat_dict = {
    f.replace('_concat.mp4', ''): os.path.join(base_output_dir, f) 
    for f in concat_file_list
}

view_videos = {
    'video_mix': concat_dict['video_mix'],  
    'plot': concat_dict['plot'],    
    'joint_Hips': concat_dict['joint_Hips'],
    'drum_dot': concat_dict['drum_dot'],
    'dance_dot': concat_dict['dance_dot'],
    
    'front_view': concat_dict['front_view'],
    # 'left_view': concat_dict['left_view'],
    # 'right_view': concat_dict['right_view'],
    # 'top_view': concat_dict['top_view'],
}

Creating concatenation files...
Concatenation complete: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00


### Generate Composite Video

In [8]:
# canvas_w, canvas_h = 1920, 1080

composite_layout_1 = [
    # Top row - side by side
    {'view': 'video_mix', 'x': 0, 'y': 0, 'width': 960, 'height': 540},
    {'view': 'front_view', 'x': 960, 'y': 0, 'width': 960, 'height': 540},
    
    # Bottom row - stacked vertically
    {'view': 'joint_Hips', 'x': 0, 'y': 540, 'width': 1920, 'height': 270},
    {'view': 'plot', 'x': 0, 'y': 810, 'width': 1920, 'height': 270},
]

layout_5views = [
    # Top row - side by side
    {'view': 'video_mix', 'x': 0, 'y': 0, 'width': 960, 'height': 540},
    {'view': 'front_view', 'x': 960, 'y': 0, 'width': 960, 'height': 540},
    
    # Bottom row - stacked vertically
    {'view': 'joint_Hips', 'x': 0, 'y': 540, 'width': 960, 'height': 270},
    {'view': 'plot', 'x': 0, 'y': 810, 'width': 960, 'height': 270},
    
    # {'view': 'drum_dot', 'x': 960, 'y': 540, 'width': 960, 'height': 540},
    {'view': 'drum_dot', 'x': 960, 'y': 540, 'width': 960, 'height': 270},
    {'view': 'dance_dot', 'x': 960, 'y': 810, 'width': 960, 'height': 270},
]

layout_2views = [
    # Top row 
    {'view': 'video_mix', 'x': 480, 'y': 0, 'width': 960, 'height': 540},
    {'view': 'drum_dot', 'x': 0, 'y': 540, 'width': 1920, 'height': 540}, # bottom row

]

final_out = os.path.join(base_output_dir, f"{file_name}_layout_5views.mp4")      # {start_time:.2f}_{end_time:.2f}

current_layout = layout_5views



saved_resized_dir = os.path.join(base_output_dir, "temp_resized")
os.makedirs(saved_resized_dir, exist_ok=True)

composite_video_elements = []

for video_element in current_layout:
    video_path = view_videos[video_element['view']]
    v_width, v_height = video_element['width'], video_element['height']
    x_pos_pxl, y_pos_pxl = video_element['x'], video_element['y']
    
    resized_path = resize_video(video_path, v_width, v_height, saved_resized_dir)
    
    composite_video_elements.append({"view": video_element['view'], 
                           "vid_path": resized_path,
                           "x_pos_pxl": x_pos_pxl,
                           "y_pos_pxl": y_pos_pxl,
                           })

#----------------------------- Create composite video -----------------------------
create_composite_video(composite_video_elements, final_out)

Resizing succeeded, output saved to: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\temp_resized\video_mix_concat.mp4
Resizing succeeded, output saved to: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\temp_resized\front_view_concat.mp4
Resizing succeeded, output saved to: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\temp_resized\joint_Hips_concat.mp4
Resizing succeeded, output saved to: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\temp_resized\plot_concat.mp4
Resizing succeeded, output saved to: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\temp_resized\drum_dot_concat.mp4
Resizing succeeded, output saved to: composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\temp_resized\dance_dot_concat.mp4
Video successfully created as composite_videos\BKO_E1_D1_01_Suku_40.00_46.00\BKO_E1_D1_01_Suku_layout_5views.mp4
